# Notebook 09: Session Memory

Purpose:
- Design NEEL’s session memory system
- Convert raw user activity into structured memory
- Decide what information is worth remembering
- Prepare memory payloads for downstream reasoning

NEEL does not store raw conversations as memory.

Instead, it remembers:
- Patterns
- Trends
- Summaries
- Reflections

Why:
- Raw logs are noisy
- Conversations contain repetition
- LLMs reason better with summaries
- Memory should support insight, not recall

NEEL uses three logical memory types:

1. Short-Term Session Memory
   - Recent behavior (last 7–14 days)
   - Used for daily/weekly reasoning

2. Long-Term Profile Memory
   - Stable user attributes (goals, preferences)
   - Changes slowly

3. Reflective Memory
   - Higher-level insights (monthly patterns)
   - Used for analysis and self-awareness

In [1]:
import pandas as pd

session_logs = pd.DataFrame([
    {"date": "2025-01-01", "work_minutes": 480, "sleep_minutes": 360, "exercise_minutes": 20, "productivity": 80},
    {"date": "2025-01-02", "work_minutes": 510, "sleep_minutes": 350, "exercise_minutes": 15, "productivity": 82},
    {"date": "2025-01-03", "work_minutes": 450, "sleep_minutes": 370, "exercise_minutes": 25, "productivity": 78},
    {"date": "2025-01-04", "work_minutes": 520, "sleep_minutes": 340, "exercise_minutes": 10, "productivity": 83},
    {"date": "2025-01-05", "work_minutes": 490, "sleep_minutes": 360, "exercise_minutes": 20, "productivity": 81},
])

In [2]:
def build_short_term_memory(logs: pd.DataFrame) -> dict:
    return {
        "days_observed": len(logs),
        "avg_work_minutes": round(logs["work_minutes"].mean(), 1),
        "avg_sleep_minutes": round(logs["sleep_minutes"].mean(), 1),
        "avg_exercise_minutes": round(logs["exercise_minutes"].mean(), 1),
        "mean_productivity": round(logs["productivity"].mean(), 1),
        "productivity_trend": (
            "stable"
            if logs["productivity"].std() < 3
            else "unstable"
        )
    }

short_term_memory = build_short_term_memory(session_logs)
short_term_memory

{'days_observed': 5,
 'avg_work_minutes': np.float64(490.0),
 'avg_sleep_minutes': np.float64(356.0),
 'avg_exercise_minutes': np.float64(18.0),
 'mean_productivity': np.float64(80.8),
 'productivity_trend': 'stable'}

This summary is memory because:
- It compresses multiple days into meaning
- It removes noise
- It preserves behavioral signal
- It is reusable across reasoning steps

Raw logs are data.
Summaries are memory.

In [3]:
def build_reflective_memory(summary: dict) -> str:
    insights = []

    if summary["avg_sleep_minutes"] < 360:
        insights.append("Sleep duration has been consistently low")

    if summary["avg_exercise_minutes"] < 30:
        insights.append("Physical activity levels are minimal")

    if summary["productivity_trend"] == "unstable":
        insights.append("Productivity levels fluctuate noticeably")

    if not insights:
        return "No concerning patterns detected"

    return "; ".join(insights)

reflective_memory = build_reflective_memory(short_term_memory)
reflective_memory

'Sleep duration has been consistently low; Physical activity levels are minimal'

In [4]:
long_term_profile = {
    "education": "MCA",
    "goal": "Become an ML Engineer",
    "priority": "Learning + Health",
    "preferred_style": "Cautious guidance"
}

In [5]:
memory_payload = {
    "short_term": short_term_memory,
    "reflective": reflective_memory,
    "profile": long_term_profile
}

memory_payload

{'short_term': {'days_observed': 5,
  'avg_work_minutes': np.float64(490.0),
  'avg_sleep_minutes': np.float64(356.0),
  'avg_exercise_minutes': np.float64(18.0),
  'mean_productivity': np.float64(80.8),
  'productivity_trend': 'stable'},
 'reflective': 'Sleep duration has been consistently low; Physical activity levels are minimal',
 'profile': {'education': 'MCA',
  'goal': 'Become an ML Engineer',
  'priority': 'Learning + Health',
  'preferred_style': 'Cautious guidance'}}

The LLM will receive:
- Aggregated summaries
- Reflections
- Stable profile context

The LLM will NOT receive:
- Raw logs
- Exact timestamps
- Individual session details
- Authentication data

The Supervisor will use memory to:
- Validate data sufficiency
- Detect instability
- Assign confidence
- Detect conflicts over time

Memory rules in NEEL:

- Memory must be structured
- Memory must be summarized
- Memory must be auditable
- Memory must support reasoning
- Memory must decay over time

If memory does not help reasoning, it should not be stored.

Session Memory in NEEL is not recall-based.

It is insight-based.

By separating memory from reasoning, NEEL:
- Avoids hallucination
- Improves reasoning quality
- Preserves user trust
- Scales safely

This memory layer is foundational for LangChain and LangGraph integration.